In [ ]:
# importing necessary Python libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import plotly.offline as pyoff
import plotly.graph_objs as go 
#import plotly.figure_factory as ff
import math
from tqdm import tqdm
# avoid displaying warnings
import warnings
warnings.filterwarnings("ignore")
import dill
import csv
#import machine learning related libraries
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.cluster import KMeans
import xgboost as xgb
import time 
order_df ,order_df_split_1m,order_df_original= None,None,None
order_df_with_all_data = None
submission_df = None
frequency_knn_cluster,recency_knn_cluster,quantity_knn_cluster = None,None,None
frequency_knn_cluster_item,recency_knn_cluster_item,quantity_knn_cluster_item = {},{},{}
dataframe_items_clusters_frequency = {}
dataframe_items_clusters_recency = {}
dataframe_items_clusters_quantity = {}
dataframe_items = {}
datafrane_items_realdataframe={}
models = []
models.append(("LR",LogisticRegression()))
models.append(("NB",GaussianNB()))
models.append(("RF",RandomForestClassifier()))
models.append(("SVC",SVC()))
models.append(("Dtree",DecisionTreeClassifier()))
models.append(("XGB",xgb.XGBClassifier(enable_categorical=True)))
models.append(("KNN",KNeighborsClassifier()))

In [ ]:
def addRecency(order_df_complete):
    global dataframe_items_clusters_recency
    [order_df,item] = order_df_complete
    ctm_max_purchase = order_df.groupby('userID').date.max().reset_index()
    ctm_max_purchase.columns = ['userID','MaxPurchaseDate']
    ctm_max_purchase['Recency'] = (ctm_max_purchase['MaxPurchaseDate'].max() - ctm_max_purchase['MaxPurchaseDate']).dt.days
    number_of_clusters = 3
    recency_knn_cluster = KMeans(n_clusters=number_of_clusters)
    recency_knn_cluster.fit(ctm_max_purchase[['Recency']])
    ctm_max_purchase['RecencyCluster'] = recency_knn_cluster.predict(ctm_max_purchase[['Recency']])
    order_df = pd.merge(order_df, ctm_max_purchase[['userID', 'RecencyCluster']], on='userID')
    dataframe_items_clusters_recency[item] = recency_knn_cluster
    return [order_df,item]
def addCompleteRecency(order_df):
    ctm_max_purchase = order_df.groupby('userID').date.max().reset_index()
    ctm_max_purchase.columns = ['userID','MaxPurchaseDate']
    ctm_max_purchase['Recency'] = (ctm_max_purchase['MaxPurchaseDate'].max() - ctm_max_purchase['MaxPurchaseDate']).dt.days
    number_of_clusters = 3
    recency_knn_cluster = KMeans(n_clusters=number_of_clusters)
    recency_knn_cluster.fit(ctm_max_purchase[['Recency']])
    ctm_max_purchase['RecencyCluster'] = recency_knn_cluster.predict(ctm_max_purchase[['Recency']])
    order_df = pd.merge(order_df, ctm_max_purchase[['userID', 'RecencyCluster']], on='userID')
    return order_df

In [ ]:
def addFrequency(order_df_complete):
    global dataframe_items_clusters_frequency
    [order_df,item] = order_df_complete
    ctm_frequency = order_df.groupby('userID').date.count().reset_index()
    ctm_frequency.columns = ['userID', 'Frequency']
    frequency_knn_cluster = KMeans(n_clusters=3)
    frequency_knn_cluster.fit(ctm_frequency[['Frequency']])
    ctm_frequency['FrequencyCluster'] = frequency_knn_cluster.predict(ctm_frequency[['Frequency']])
    order_df = pd.merge(order_df, ctm_frequency[['userID', 'FrequencyCluster']], on='userID')
    dataframe_items_clusters_frequency[item] = frequency_knn_cluster
    return [order_df,item]
def addCompleteFrequency(order_df):
    ctm_frequency = order_df.groupby('userID').date.count().reset_index()
    ctm_frequency.columns = ['userID', 'Frequency']
    frequency_knn_cluster = KMeans(n_clusters=3)
    frequency_knn_cluster.fit(ctm_frequency[['Frequency']])
    ctm_frequency['FrequencyCluster'] = frequency_knn_cluster.predict(ctm_frequency[['Frequency']])
    order_df = pd.merge(order_df, ctm_frequency[['userID', 'FrequencyCluster']], on='userID')
    return order_df

In [ ]:
def addQuantity(order_df_complete):
    [order_df,item] = order_df_complete
    ctm_total_quantity = order_df.groupby('userID').order.sum().reset_index()
    ctm_total_quantity.columns = ['userID','Quantity_Total']
    quantity_knn_cluster = KMeans(n_clusters=3)
    quantity_knn_cluster.fit(ctm_total_quantity[['Quantity_Total']])
    ctm_total_quantity['Quantity_Cluster'] = quantity_knn_cluster.predict(ctm_total_quantity[['Quantity_Total']])
    order_df = pd.merge(order_df, ctm_total_quantity[['userID', 'Quantity_Cluster']], on='userID')
    global dataframe_items_clusters_quantity
    dataframe_items_clusters_quantity[item] = quantity_knn_cluster
    return [order_df,item]
def addCompleteQuantity(order_df):
    ctm_total_quantity = order_df.groupby('userID').order.sum().reset_index()
    ctm_total_quantity.columns = ['userID','Quantity_Total']
    quantity_knn_cluster = KMeans(n_clusters=3)
    quantity_knn_cluster.fit(ctm_total_quantity[['Quantity_Total']])
    ctm_total_quantity['Quantity_Cluster'] = quantity_knn_cluster.predict(ctm_total_quantity[['Quantity_Total']])
    order_df = pd.merge(order_df, ctm_total_quantity[['userID', 'Quantity_Cluster']], on='userID')
    return order_df

In [ ]:
def set_variables():
    global order_df_original,submission_df,order_df,order_df_split_1m,order_df_with_all_data
    order_df_original = pd.read_csv('../src/data/orders_before_dec.csv',delimiter='|',parse_dates=True)
    order_df_original['date'] = pd.to_datetime(order_df_original['date'], errors='coerce')
    order_df_original['week'] = order_df_original.date.dt.isocalendar().week
    order_df_original['week'] = order_df_original['week']%4
    submission_df = pd.read_csv('../src/data/submission_dec.csv',delimiter='|',parse_dates=True)
    order_df_with_all_data = addCompleteRecency(addCompleteQuantity(addCompleteFrequency(order_df_original)))
set_variables()

In [ ]:
order_df_with_all_data.to_csv('completedataset_user_embedding_freq-rec-uency&qunatity.csv')

In [ ]:
order_df_original

In [ ]:
for idx,row in submission_df.iterrows():
    df1 = order_df_original[order_df_original['userID']==row['userID']]
    df2 = df1[df1['itemID']==row['itemID']]
    if df2.empty:
        print(f"user -{row['userID']} AND item-{row['itemID']} does not exist")

In [ ]:
def split_df_onemonth(df):
    df['month'] = df.date.dt.month
    df_big = df[df['month']<df['date'].max().month]
    df_small = df[df['month']==df['date'].max().month]
    df_big.drop(['month'],1,inplace=True)
    df_small.drop(['month'],1,inplace=True)
    return df_big,df_small

In [ ]:
def get_itemid_from_submission():
    global submission_df
    items = submission_df.itemID.unique()
    return items

In [ ]:
def build_dataframe_for_item(itemid):
    global order_df_original,datafrane_items_realdataframe
    temp_order_df = order_df_original[order_df_original['itemID']==itemid]
    temp_order_df.drop(['itemID'],axis=1,inplace=True)
    big_df ,small_df = split_df_onemonth(temp_order_df)
    big_df = addRecency(addQuantity(addFrequency([big_df,itemid])))[0]
    datafrane_items_realdataframe[itemid]=big_df
    return big_df,small_df

In [ ]:
def add_y_label_to_dataframe(a,b):
    #create a dataframe with customer id and first purchase date in tx_next
    tx_next_first_purchase = b.groupby('userID').date.min().reset_index()
    tx_next_first_purchase.columns = ['userID','MinPurchaseDate']

    #create a dataframe with customer id and last purchase date in tx_5m
    tx_last_purchase = a.groupby('userID').date.max().reset_index()
    tx_last_purchase.columns = ['userID','MaxPurchaseDate']

    #merge two dataframes
    tx_purchase_dates = pd.merge(tx_last_purchase,tx_next_first_purchase,on='userID',how='left')

    #calculate the time difference in days:
    tx_purchase_dates['NextPurchaseDay'] = (tx_purchase_dates['MinPurchaseDate'] - tx_purchase_dates['MaxPurchaseDate']).dt.days
    tx_user = pd.DataFrame(a['userID'].unique())
    tx_user.columns = ['userID']
    #merge with tx_user 
    tx_user = pd.merge(tx_user, tx_purchase_dates[['userID','NextPurchaseDay']],on='userID',how='left')
    a = pd.merge(a,tx_user,on='userID')
    a['NextPurchaseDayRange'] = 1
    a.loc[a.NextPurchaseDay>7,'NextPurchaseDayRange'] = 2
    a.loc[a.NextPurchaseDay>14,'NextPurchaseDayRange'] = 3
    a.loc[a.NextPurchaseDay>21,'NextPurchaseDayRange'] = 4
    a.loc[a.NextPurchaseDay>28,'NextPurchaseDayRange'] = 0
    a.drop(['NextPurchaseDay'],1,inplace=True)
    return a

In [ ]:
def build_train_test_set(a):
    X, y = a.drop('NextPurchaseDayRange',axis=1), a.NextPurchaseDayRange
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X,y,X_train, X_test, y_train, y_test

In [ ]:
def train_best_model(model,X,y):
    if model == 'LR':
        reg = LogisticRegression().fit(X, y)
    if model == 'NB':
        reg = GaussianNB().fit(X, y)
    if model == 'RF':
        reg = RandomForestClassifier().fit(X, y)
    if model == 'SVC':
        reg = SVC().fit(X, y)
    if model == 'Dtree':
        reg = DecisionTreeClassifier().fit(X, y)
    if model == 'XGB':
        reg = xgb.XGBClassifier(enable_categorical=True).fit(X, y)
    if model == 'KNN':
        reg = KNeighborsClassifier().fit(X, y)
    return reg 

In [ ]:
def find_best_model(X,y,X_train, X_test, y_train, y_test):
    dic = {}
    global models
    for name,model in models:
        kfold = KFold(n_splits=2)
        cv_result = cross_val_score(model,X,y, cv = kfold,scoring = "accuracy")
        if cv_result[0]!=math.nan :
            dic[name]=cv_result[0]
        # print(name, cv_result[0])
    if len(dic) != 0:
        bestmodel = max(dic, key=dic.get)
        return train_best_model(bestmodel,X,y),bestmodel
    else:
        return None,None

In [ ]:
def get_user_representation_from_complete(userid,itemid):
    global order_df_with_all_data
    df = order_df_with_all_data[order_df_with_all_data['userID']==userid].drop(['userID','date','week','itemID'],1).iloc[0]
    return df
    

In [ ]:
def get_user_representation_from_item(userid,itemid):
    global datafrane_items_realdataframe
    p = datafrane_items_realdataframe[itemid]
    df = p[p['userID']==userid].drop(['userID','date','week'],1).iloc[0]
    return df

In [ ]:
def get_models_for_each_item():
    for item in get_itemid_from_submission():
        try:
            a,b=build_dataframe_for_item(item)
            a = add_y_label_to_dataframe(a,b)
            X,y,X_train, X_test, y_train, y_test = build_train_test_set(a)
            model,bestmodel = find_best_model(X.drop(['userID','date','week'],1),y,X_train.drop(['userID','date','week'],1), X_test.drop(['userID','date','week'],1), y_train, y_test)
            if model == None:
                dataframe_items[item] = 'no best model'
            else:
                dataframe_items[item] = [bestmodel,model]
        except: 
            dataframe_items[item] = 'error in splitting data'

In [ ]:
# import dill
# dill.dump_session('notebook_env.db')
# dill.load_session('notebook_env.db')

In [ ]:
def fill_csv_with_complete_data():
    get_models_for_each_item()
    with open(r"mysub_complete.csv", "w") as csv_file:
        writer = csv.writer(csv_file, delimiter ="|",quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["userID","itemID","prediction"])
        for idx,row in submission_df.iterrows():
            user_rep = get_user_representation_from_complete(row['userID'],row['itemID'])
            c = dataframe_items[int(row['itemID'])][1]
            if type(c)==str:
                writer.writerow([int(row['userID']),int(row['itemID']),"None"])
            else:
                try:
                    d = c.predict([user_rep])
                    writer.writerow([int(row['userID']),int(row['itemID']),d[0]])
                except:
                    print(f"error in {dataframe_items[int(row['itemID'])][0]} ")
                    writer.writerow([row['userID'],row['itemID'],"INTERNAL ERROR"])

In [ ]:
def fill_csv_from_item_data():
    #get_models_for_each_item()
    with open(r"mysub_item.csv", "w") as csv_file:
        writer = csv.writer(csv_file, delimiter ="|",quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["userID","itemID","prediction"])
        for idx,row in submission_df.iterrows():
            try:
                user_rep = get_user_representation_from_item(row['userID'],row['itemID'])
            except:
                print(f"User rep does not exist for {row['itemID']}")
                writer.writerow([row['userID'],row['itemID'],"User rep couldnt be found"])
            c = dataframe_items[int(row['itemID'])][1]
            if type(c)==str:
                writer.writerow([int(row['userID']),int(row['itemID']),"None"])
            else:
                try:
                    d = c.predict([user_rep])
                    writer.writerow([int(row['userID']),int(row['itemID']),d[0]])
                except:
                    print(f"error in {dataframe_items[int(row['itemID'])][0]} ")
                    writer.writerow([row['userID'],row['itemID'],"Model doesnt exist"])

In [ ]:
fill_csv_with_complete_data()
fill_csv_from_item_data()

Analysis of submission

In [ ]:
com_sub_dec =  pd.read_csv('mysub_complete_dec.csv',delimiter='|')
item_sub_dec =  pd.read_csv('mysub_item_dec.csv',delimiter='|')
com_sub_jan =  pd.read_csv('mysub_fromcomplete_jan.csv',delimiter='|')
item_sub_jan =  pd.read_csv('mysub_fromitem_jan.csv',delimiter='|')
dec_pujit =  pd.read_csv('submission_dec_01_pujit.csv',delimiter='|')
jan_pujit =  pd.read_csv('submission_jan_01_pujit.csv',delimiter='|')

In [ ]:
item_sub_dec = item_sub_dec[item_sub_dec['prediction']!='User rep couldnt be found']
item_sub_jan = item_sub_jan[item_sub_jan['prediction']!='User rep couldnt be found']

In [ ]:
item_sub_dec

In [ ]:
dec_pujit

In [ ]:
com_sub_dec['prediction'].iloc[1] = None

In [ ]:
for idx,row in dec_pujit.iterrows():
    try:
        int(com_sub_dec.iloc[idx]['prediction'])
    except:
        com_sub_dec['prediction'].iloc[idx] = int(row['prediction'])
    try:
        int(item_sub_dec['prediction'].iloc[idx])
    except:
        item_sub_dec['prediction'].iloc[idx] = int(row['prediction'])

for idx,row in jan_pujit.iterrows():
    try:
        int(com_sub_jan['prediction'].iloc[idx])
    except:
        com_sub_jan['prediction'].iloc[idx] = int(row['prediction'])
    try:
        int(item_sub_jan['prediction'].iloc[idx])
    except:
        item_sub_jan['prediction'].iloc[idx] = int(row['prediction'])

In [ ]:
for idx,row in dec_pujit.iterrows():
    if np.isnan(item_sub_dec.iloc[idx]['prediction']):
        print('here')
        item_sub_dec['prediction'].iloc[idx] = int(row['prediction'])
for idx,row in jan_pujit.iterrows():
    if np.isnan(item_sub_jan.iloc[idx]['prediction']):
        item_sub_jan['prediction'].iloc[idx] = int(row['prediction'])

In [ ]:
for idx,row in dec_pujit.iterrows():
    print(item_sub_dec['prediction'].iloc[idx])

In [ ]:
print(com_sub_dec.prediction.unique())
print(item_sub_dec.prediction.unique())
print(com_sub_jan.prediction.unique())
print(item_sub_jan.prediction.unique())


In [ ]:
com_sub_dec['prediction']=com_sub_dec['prediction'].astype(int)
com_sub_dec['userID']=com_sub_dec['userID'].astype(int)
com_sub_dec['itemID']=com_sub_dec['itemID'].astype(int)

com_sub_jan['prediction']=com_sub_jan['prediction'].astype(int)
com_sub_jan['userID']=com_sub_jan['userID'].astype(int)
com_sub_jan['itemID']=com_sub_jan['itemID'].astype(int)

item_sub_dec['prediction']=item_sub_dec['prediction'].astype(int)
item_sub_dec['userID']=item_sub_dec['userID'].astype(int)
item_sub_dec['itemID']=item_sub_dec['itemID'].astype(int)

item_sub_jan['prediction']=item_sub_jan['prediction'].astype(int)
item_sub_jan['userID']=item_sub_jan['userID'].astype(int)
item_sub_jan['itemID']=item_sub_jan['itemID'].astype(int)

In [ ]:
item_sub_jan['prediction']=pd.to_numeric(item_sub_jan['prediction'], errors='coerce')
item_sub_jan['userID']=item_sub_jan['userID'].astype(int)
item_sub_jan['itemID']=item_sub_jan['itemID'].astype(int)

In [ ]:
item_sub_dec['prediction']=pd.to_numeric(item_sub_dec['prediction'], errors='coerce')
item_sub_dec['userID']=item_sub_dec['userID'].astype(int)
item_sub_dec['itemID']=item_sub_dec['itemID'].astype(int)

In [ ]:
com_sub_dec.to_csv('cd.csv',index=False,sep='|')
com_sub_jan.to_csv('cj.csv',index=False,sep='|')
item_sub_jan.to_csv('ij.csv',index=False,sep='|')
item_sub_dec.to_csv('id.csv',index=False,sep='|')